In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path = 'nietzsche.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

#     start_index = random.randint(0, len(text) - maxlen - 1)

#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print()
#         print('----- diversity:', diversity)

#         generated = ''
#         sentence = text[start_index: start_index + maxlen]
#         generated += sentence
#         print('----- Generating with seed: "' + sentence + '"')
#         sys.stdout.write(generated)

#         for i in range(400):
#             x = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(sentence):
#                 x[0, t, char_indices[char]] = 1.

#             preds = model.predict(x, verbose=0)[0]
#             next_index = sample(preds, diversity)
#             next_char = indices_char[next_index]

#             generated += next_char
#             sentence = sentence[1:] + next_char

#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print()


Using TensorFlow backend.


corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...

--------------------------------------------------
Iteration 0
Epoch 1/1
200285/200285 [==============================] - 248s - loss: 1.9789   

--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 247s - loss: 1.6278   

--------------------------------------------------
Iteration 2
Epoch 1/1
200285/200285 [==============================] - 254s - loss: 1.5381   

--------------------------------------------------
Iteration 3
Epoch 1/1
200285/200285 [==============================] - 252s - loss: 1.4946   

--------------------------------------------------
Iteration 4
Epoch 1/1
200285/200285 [==============================] - 247s - loss: 1.4657   

--------------------------------------------------
Iteration 5
Epoch 1/1
200285/200285 [==============================] - 250s - loss: 1.4445   

--------------------------------

In [2]:
model.save('keras_model.h5')

In [3]:
from keras.models import load_model

model = load_model('keras_model.h5')

In [30]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(preds)
#     return np.argmax(probas)

In [43]:
start_index = random.randint(0, len(text) - maxlen - 1)

# for diversity in [0.2, 0.5, 1.0, 1.2]:
for diversity in [0.2]:
#     print()
#     print('----- diversity:', diversity)

    generated = ''
#     sentence = "ally "
    sentence = text[start_index: start_index + maxlen]
#     sentence = "smile it"
#     print(sentence)
#     print()
    original_sentence = sentence
    generated += sentence
#     print('----- Generating with seed: "' + sentence + '"')
#     sys.stdout.write(generated)

    for i in range(50):
#     while True:
        x = np.zeros((1, maxlen, len(chars)))

        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
#         break
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
        
#         if len(sentence) + 2 > len(original_sentence) and next_char == ' ':
#             break
    print()

re the superior of the superior of the superior of
